In [2]:
import sys; sys.path.append("../")
from feature_extractor import FeatureExtractor
import torch
import numpy as np
import time
from cascade import CascadeClassifier
import cv2

In [3]:
torch.cuda.is_available()

True

In [4]:
parent = '../hFeatures3/'
data_path = 'new_data3'

# Features Extraction

In [5]:
mean = 111.02404842061983
std = 56.926623499738575

In [25]:
def transform1(img):
  # return img
  img = (img - img.mean()) / (img.std() + 1e-8)
  return cv2.normalize(img, None, alpha=0, beta=255, norm_type=cv2.NORM_MINMAX, dtype=cv2.CV_8U)

In [26]:

feature_extractor = FeatureExtractor(shape=(19,19), 
                                     percentile=20,
                                     all_features_file=parent + "all_features.npz",
                                     selected_features_file=parent + "selected_features.npz",
                                     labels_file=parent + "labels.npy",
                                     indecies_file=parent + "indecies.npy",)

In [27]:

feature_extractor.extractFeatures(pos_path='../'+data_path+'/train/face', 
                                  neg_path='../'+data_path+'/train/non-face',
                                  transform=transform1,)
print('done')


all_features shape: (63960, 10826)
Now, saving all features to ../hFeatures3/all_features.npz
done


In [28]:
# del feature_extractor.f2
# del feature_extractor.f3
# del feature_extractor.f4

In [29]:
indecies, X = feature_extractor.selectPercentile()
y = np.load(parent + 'labels.npy')
print(X.shape, y.shape)
print('done')


(12792, 10826) (10826,)
done


In [30]:
X.shape

(12792, 10826)

In [31]:
# temp = np.load('../hFeatures3/all_features.npz')['arr_0']
temp = X
temp 

array([[-192.,   29.,   13., ...,    5.,    2.,    0.],
       [-172.,   92.,   10., ...,   24.,   17.,   -2.],
       [  19.,   -7., -170., ...,    2.,    3.,    2.],
       ...,
       [ -74.,  -62.,   -1., ...,    2.,    0.,    1.],
       [ -14., -106.,   38., ...,    0.,    0.,    0.],
       [  12.,  -57.,   -1., ...,    1.,    0.,    0.]], dtype=float32)

In [32]:
print(temp.min())
print(temp.max())
print(temp[3])
print(temp[4])

-30493.0
28166.0
[-164.  170.  -15. ...  124.   39.   -2.]
[ -25.  -30. -232. ...    3.    4.    2.]


# Training

In [40]:
%rm StrongClassifier/lastSC.last
s_t = time.time()
cascadeClassifier= CascadeClassifier(X, y, batchsize=1000, verbose=True, layers=[350])
tr_acc = cascadeClassifier.train()

cascadeClassifier.save(parent + 'cascadeClassifier.pkl')

print('accuracy: ', tr_acc)
print('time taken: ', time.time() - s_t, 's')
print('done')

$$$$$$$ Training layer 1 / 1 $$$$$$$
Starting to choose classifier
Finished training weak classifier 1 / 350
Starting to choose classifier
Finished training weak classifier 2 / 350
Starting to choose classifier
Finished training weak classifier 3 / 350
Starting to choose classifier
Finished training weak classifier 4 / 350
Starting to choose classifier
Finished training weak classifier 5 / 350
Starting to choose classifier
Finished training weak classifier 6 / 350
Starting to choose classifier
Finished training weak classifier 7 / 350
Starting to choose classifier
Finished training weak classifier 8 / 350
Starting to choose classifier
Finished training weak classifier 9 / 350
Starting to choose classifier
Finished training weak classifier 10 / 350
Starting to choose classifier
Finished training weak classifier 11 / 350
Starting to choose classifier
Finished training weak classifier 12 / 350
Starting to choose classifier
Finished training weak classifier 13 / 350
Starting to choose clas

# Testing

In [41]:
# accuracy imports
from sklearn.metrics import accuracy_score, f1_score, precision_score, recall_score, classification_report, confusion_matrix


In [42]:
s_t = time.time()
t_f_idx_map, t_features, t_labels = feature_extractor.extractFeaturesByIndecies(pos_path='../'+data_path+'/test/face', 
                                            neg_path='../'+data_path+'/test/non-face',
                                            cascadeClassifier=cascadeClassifier,
                                            transform=transform1)
print(t_features.shape, t_labels.shape)
predictions = cascadeClassifier.predict(t_features, t_f_idx_map)
print('test accuracy: ', np.sum(predictions == t_labels) / t_labels.shape[0])
print('time taken: ', time.time() - s_t, 's')

Now reading dataset...
Now extracting features from dataset...
(323, 2715) (2715,)
test accuracy:  0.9406998158379374
time taken:  1.0659596920013428 s


In [43]:
# report
print('accuracy: ', accuracy_score(t_labels, predictions))
print('f1 score macro: ', f1_score(t_labels, predictions, average='macro'))
print('f1 score binary: ', f1_score(t_labels, predictions))
print('f1 score micro: ', f1_score(t_labels, predictions, average='micro'))
print('precision: ', precision_score(t_labels, predictions))
print('recall: ', recall_score(t_labels, predictions))
print('confusion matrix: \n', confusion_matrix(t_labels, predictions))
print('classification report: \n', classification_report(t_labels, predictions, digits=4))


accuracy:  0.9406998158379374
f1 score macro:  0.9350615972575571
f1 score binary:  0.9159268929503916
f1 score micro:  0.9406998158379374
precision:  0.8683168316831683
recall:  0.969060773480663
confusion matrix: 
 [[1677  133]
 [  28  877]]
classification report: 
               precision    recall  f1-score   support

           0     0.9836    0.9265    0.9542      1810
           1     0.8683    0.9691    0.9159       905

    accuracy                         0.9407      2715
   macro avg     0.9259    0.9478    0.9351      2715
weighted avg     0.9452    0.9407    0.9414      2715

